# Word Embeddings : le modèle Word2Vec

## Imports

In [1]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [11]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [12]:
infile = f"../data/tmp/1958_clean.txt"
sentences = MySentences(infile)

In [13]:
print(sentences)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [14]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [15]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [16]:
len(bigram_phrases.vocab.keys())

64803

Prenons une clé au hasard :

In [17]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

pompiers_ecoles


Le dictionnaire indique le score de cette coocurrence :

In [18]:
bigram_phrases.vocab[key_]

1

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

In [19]:
%time bigram_phrases[sentences]

CPU times: user 499 ms, sys: 846 µs, total: 500 ms
Wall time: 512 ms


### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [20]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

In [21]:
%time bigram_phraser[sentences]

CPU times: user 446 ms, sys: 8.78 ms, total: 454 ms
Wall time: 465 ms


### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [22]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [23]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [24]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [25]:
print(corpus[:100])

[['ville_bruxelles', 'rapport_annuel', 'presente', 'conseil', 'seance', 'bourgmestre_echevins', 'execution', 'article', 'loi', 'mars', 'imprimerie', 'schaumans', 'parvis', 'saint', 'gilles', 'bruxelles', 'chapitre', 'administration', 'chapitre', 'administration', 'conseil', 'reuni', 'cours_annee', 'tenu', 'seances', 'sections', 'reunies', 'seances', 'sections', 'seances', 'commissions', 'comites', 'visite', 'chantiers', 'internationale', 'theatre', 'fondations', 'reunies', 'cite', 'jouet', 'rey', 'juliette', 'inaugure', 'brigade', 'canine', 'police', 'locaux', 'service', 'ves', 'assiste', 'essais', 'carillon', 'electronique', 'hotel', 'annee', 'college', 'reuni', 'pris', 'decisions', 'nombre', 'agents', 'fonctions', 'administration', 'centrale', 'services', 'exterieurs', 'dependent', 'agents', 'effectifs', 'agents', 'temporaires', 'police', 'fiq', 'iii', 'sapeurs_pompiers', 'ecoles', 'personnel', 'salarie', 'concierges', 'domaine', 'prive', 'vii', 'nettoyeuses', 'femmes_service', 'tota

## Entrainement d'un modèle Word2Vec sur ce corpus

In [26]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=7, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=10 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 433 ms, sys: 0 ns, total: 433 ms
Wall time: 459 ms


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [27]:
outfile = f"../data/1958.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [28]:
model = Word2Vec.load("../data/bulletins.model")

### Imprimer le vecteur d'un terme

In [29]:
model.wv["bruxelles"]

array([ 0.03666609, -0.10040963,  0.03654497,  0.09746505,  0.03343763,
       -0.03119738, -0.03212402,  0.02686526, -0.08945952,  0.0548064 ,
        0.04578976, -0.0410102 ,  0.04764709, -0.01300347,  0.02187117,
       -0.02530024, -0.02506259,  0.02995131,  0.01435767,  0.0954808 ,
       -0.00400413,  0.06046201,  0.01500096, -0.00477403,  0.02370468,
        0.09968854, -0.01848824,  0.05762551, -0.06189843, -0.00792185,
        0.05806829,  0.0392398 ], dtype=float32)

### Calculer la similarité entre deux termes

In [30]:
model.wv.similarity("eleve", "ecole")

-0.04627937

In [31]:
model.wv.similarity("ecole", "enseignement")

0.14080766

In [32]:
model.wv.similarity("eleves", "enseignement")

0.10957636

In [33]:
model.wv.similarity("eleve", "andre")

0.5452789

### Chercher les mots les plus proches d'un terme donné

In [34]:
model.wv.most_similar("eleve", topn=10)

[('andre', 0.5452788472175598),
 ('nombre_total', 0.5224970579147339),
 ('accord', 0.49939364194869995),
 ('voir', 0.48816365003585815),
 ('expropriation', 0.4870023727416992),
 ('geographie', 0.48583516478538513),
 ('malades', 0.48367902636528015),
 ('administration_communale', 0.479801207780838),
 ('bois', 0.46865031123161316),
 ('admission', 0.4654456079006195)]

In [35]:
model.wv.most_similar("andre", topn=10)

[('tuyaux', 0.628588855266571),
 ('reuni', 0.5898244976997375),
 ('experience', 0.5628107786178589),
 ('eleve', 0.5452788472175598),
 ('relative', 0.5288161039352417),
 ('habitants', 0.5254650712013245),
 ('registres_population', 0.5208917260169983),
 ('etienne', 0.5188571810722351),
 ('reparation', 0.5174610614776611),
 ('annees', 0.5140024423599243)]

In [36]:
model.wv.most_similar("enseignement", topn=10)

[('mois', 0.6234209537506104),
 ('exposition', 0.5855048298835754),
 ('bourgmestre', 0.5816928148269653),
 ('plafonnage', 0.579774796962738),
 ('effectifs', 0.5771617293357849),
 ('suppression', 0.5762450098991394),
 ('nes', 0.5718750357627869),
 ('belgique', 0.5703576803207397),
 ('fevrier', 0.5668333172798157),
 ('publique', 0.5484593510627747)]

### Faire des recherches complexes à travers l'espace vectoriel

In [37]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('utilisation', 0.5564562678337097), ('eclairage', 0.5360033512115479), ('centre_sante', 0.5359033346176147), ('administratif', 0.5171180963516235), ('defauts', 0.5048459768295288), ('quartier', 0.5017017126083374), ('aupres', 0.4993170499801636), ('quitte', 0.4975280165672302), ('chiffre', 0.4873804450035095), ('esprit', 0.4821018576622009)]


In [38]:
print(model.wv.most_similar(positive=['cours', 'ecole'], negative=['bruxelles']))

[('grands_travaux', 0.5812001824378967), ('jonction', 0.5400947332382202), ('full_time', 0.5200813412666321), ('louvain', 0.5037940740585327), ('brevet', 0.5007188320159912), ('suivent', 0.49671670794487), ('cours_annee_scolaire', 0.4873899519443512), ('harmonie', 0.4728219509124756), ('possede', 0.4692592918872833), ('union', 0.4506838023662567)]
